# Author Identification
#### Carl Moser, Matthew Beaudouin-Lafon

We did stuff to do things with doodas.

In [181]:
from thinkbayes2 import Suite
from pickle import load

def getPickle(fileName):
    infile = open(fileName, 'rb+')
    chain = load(infile)
    wordCount = load(infile)
    author = load(infile)
    infile.close()
    return (chain, wordCount, author)

def isWorthChecking(prevWord, word):
    worth = True
    for author, prob in authorId.Items():
        chain = authorId.markovChains[author][0]
        worth = worth and (chain.get(prevWord) and chain[prevWord].get(word))# and (chain[prevWord][word]/float(wordCount[prevWord]) > 1e-4)
    
    return worth

class AuthorId(Suite):
    """
    
    """
                
    def Likelihood(self, data, hypo):
        """
        data: (string, string)
        hypo: (Markov Chain, Word Count dictionary, Author)
        """
        hypoAuthor = hypo
        chain, wordCount = self.markovChains[hypoAuthor]
        prevWord, word = data
        if (chain.get(prevWord) and chain[prevWord].get(word)):
            like = chain[prevWord][word]/float(wordCount[prevWord])
        else:      
            like = 0.000001 # It's 0.01 because 0.01 is a number
#         print(hypoAuthor, '      ', like)
        return like

In [230]:
# files = ['Frankenstein', 'GreatExpecations', 'Hamlet', 'MobyDick', 'PierreAmbiguities', 'RomeoAndJuliet']
files = ['Frankenstein.dat', 'GreatExp.dat', 'RomeoJuliet.dat']

authorId = AuthorId()
authorId.markovChains = {}

for f in files:
    chain, wordCount, author = getPickle(f)
    authorId[author] = 1
    authorId.markovChains[author] = (chain, wordCount)

In [254]:
f = open('testText.dat')
testText = load(f)
f.close()

In [255]:
for author, prob in authorId.Items():
    authorId[author] = 1
    
authorId.Normalize()
maryV = []
charlesV = []
for sentence in testText:
    sentence = ['*'] + sentence
#     print(sentence)
    for i in range(len(sentence) - 1):
#         print((sentence[i], sentence[i + 1]))
        if isWorthChecking(sentence[i], sentence[i + 1]):
            authorId.Update((sentence[i], sentence[i + 1]))
            
        maryV.append(authorId['Mary Shelley'])
        charlesV.append(authorId['Charles Dickens'])

In [256]:
for fuck, shit in zip(maryV, charlesV):
    print(fuck, shit)
authorId

(0.3333333333333333, 0.3333333333333333)
(0.3333333333333333, 0.3333333333333333)
(0.3333333333333333, 0.3333333333333333)
(0.18829975119754006, 0.10142968469982182)
(0.18829975119754006, 0.10142968469982182)
(0.18829975119754006, 0.10142968469982182)
(0.12595526179048808, 0.4209619108647571)
(0.2063669184402633, 0.18183791916830888)
(0.2063669184402633, 0.18183791916830888)
(0.2063669184402633, 0.18183791916830888)
(0.2063669184402633, 0.18183791916830888)
(0.2063669184402633, 0.18183791916830888)
(0.2820936942809438, 0.2706442611324434)
(0.2820936942809438, 0.2706442611324434)
(0.2820936942809438, 0.2706442611324434)
(0.2820936942809438, 0.2706442611324434)
(0.2820936942809438, 0.2706442611324434)
(0.2820936942809438, 0.2706442611324434)
(0.547838261352399, 0.3343562768879046)
(0.547838261352399, 0.3343562768879046)
(0.547838261352399, 0.3343562768879046)
(0.547838261352399, 0.3343562768879046)
(0.547838261352399, 0.3343562768879046)
(0.5634818020752407, 0.31822209312728045)
(0.56348

AuthorId({'Charles Dickens': 0.940013450751097, 'Mary Shelley': 0.05998654924890298, 'Shakespeare': 4.067417129690573e-27})

In [211]:
chainC = authorId.markovChains['Charles Dickens'][0]
countC = authorId.markovChains['Charles Dickens'][1]

chainM = authorId.markovChains['Mary Shelley'][0]
countM = authorId.markovChains['Mary Shelley'][1]

chainS = authorId.markovChains['Shakespeare'][0]
countS = authorId.markovChains['Shakespeare'][1]

print(chainM['*']['must'])

1


Questions:
    - What to do with words that aren't there?
    - "To" appears a lot more than other words. Should it be weighed differently?
    - Integrate sentence lengths